In [ ]:
#Section 1
#Collaborative Filtering :

import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

ratings = pd.read_csv("/content/sample_data/rating.csv")

movies = pd.read_csv("/content/sample_data/anime.csv")

n_ratings = len(ratings)
n_movies = len(ratings['anime_id'].unique())
n_users = len(ratings['user_id'].unique())

#get top videos that user has watched
def get_movies(user_id):

  select = ratings.loc[ratings['user_id'] == user_id]
  sel = select[['anime_id', 'rating']]
  sel_sorted = sel.sort_values(by = 'rating', ascending = False)

  sel_mov = sel_sorted['anime_id'].tolist()

  all_mov = sel_sorted['anime_id'].tolist()

  m_top = sel_mov[0:10]

  return m_top , all_mov
  

# create user-item matrix using scipy csr matrix
from scipy.sparse import csr_matrix

def create_matrix(df):
	
	N = len(df['user_id'].unique())
	M = len(df['anime_id'].unique())
	
	# Map Ids to indices
	user_mapper = dict(zip(np.unique(df["user_id"]), list(range(N))))
	movie_mapper = dict(zip(np.unique(df["anime_id"]), list(range(M))))
	
	# Map indices to IDs
	user_inv_mapper = dict(zip(list(range(N)), np.unique(df["user_id"])))
	movie_inv_mapper = dict(zip(list(range(M)), np.unique(df["anime_id"])))
	
	user_index = [user_mapper[i] for i in df['user_id']]
	movie_index = [movie_mapper[i] for i in df['anime_id']]

	X = csr_matrix((df["rating"], (movie_index, user_index)), shape=(M, N))
	
	return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_matrix(ratings)


from sklearn.neighbors import NearestNeighbors


#Find similar movies using KNN
def find_similar_movies(movie_id, X, k, metric='cosine', show_distance=False):
      
    neighbour_ids = []
      
    movie_ind = movie_mapper[movie_id]
    movie_vec = X[movie_ind]
    k+=1
    kNN = NearestNeighbors(n_neighbors=k, algorithm="brute", metric=metric)
    kNN.fit(X)
    movie_vec = movie_vec.reshape(1,-1)
    neighbour = kNN.kneighbors(movie_vec, return_distance=show_distance)
    for i in range(0,k):
        n = neighbour.item(i)
        neighbour_ids.append(movie_inv_mapper[n])
    neighbour_ids.pop(0)
    return neighbour_ids


movie_titles = dict(zip(movies['anime_id'], movies['name']))

print("")
print("for user id 3")
print("")

print("Suggested videos :")
print("")

top_movies , all_mov = get_movies(3)

rec_mov = []
for i in top_movies:

 similar_ids = find_similar_movies(i, X, k=5)
 for j in similar_ids:
	 if (j not in rec_mov) and (j not in all_mov):
		 	 rec_mov.append(j)

ten_mov = rec_mov[0:10]

for i in ten_mov:
	
 print(movie_titles[i])
 print("")
